# Local Optimization Example Notebook

This notebook demonstrates how to use the local optimization procedure based on the topological descriptors by Kozlov et al. and the Effective Medium Theory (EMT), as it is implemented in the NPL (Numerical Python Library). 

## Objectives
- Understand the principles behind local optimization using topological descriptors.
- Learn how to apply the EMT for local optimization tasks.
- Explore the implementation details in NPL.

## Contents
1. **Introduction to Local Optimization**: A brief overview of local optimization and its significance.
2. **Topological Descriptors by Kozlov et al.**: Explanation of the topological descriptors used in the optimization process.
3. **Effective Medium Theory (EMT)**: Introduction to EMT and its role in local optimization.
4. **Implementation in NPL**: Step-by-step guide on how to use NPL for local optimization.
5. **Examples and Applications**: Practical examples demonstrating the use of local optimization in various scenarios.
6. **Conclusion**: Summary of key takeaways and further reading suggestions.

## Prerequisites
- Basic understanding of optimization techniques.
- Familiarity with Python programming.
- Knowledge of numerical libraries such as NumPy.

## Getting Started
To get started, ensure you have the NPL library installed. You can install it using pip:
```bash
pip install npl

In [14]:
import npl.core.nanoparticle as NP
import npl.optimization.go_search as GOS
from npl.optimization.local_optimization.local_optimization import local_optimization

import npl.calculators.energy_calculator as EC

from ase.visualize import view
import pickle

In [10]:
"""Create a list of particles with fixed composition and shape (truncated octahedron), but random ordering
and calculate the energy using EMT
"""

def create_octahedron_training_set(n_particles, height, trunc, stoichiometry):
    emt_calculator = EC.EMTCalculator(fmax=0.03, steps=1000)
    
    training_set = []
    for i in range(n_particles):
        p = NP.Nanoparticle()
        p.truncated_octahedron(height, trunc, stoichiometry)
        emt_calculator.compute_energy(p)
        training_set.append(p)
        
    return training_set

In [11]:
"""Create one randomly ordered start particle"""

def create_start_particle(height, trunc, stoichiometry):
    start_particle = NP.Nanoparticle()
    start_particle.truncated_octahedron(height, trunc, stoichiometry)
    return start_particle

In [12]:
"""Create the training set with 30 particles"""
stoichiometry={'Pt' : 55, 'Au' : 24}

training_set = create_octahedron_training_set(30, 5, 1, stoichiometry)

In [21]:
"""First we create an Object for a global optimization search and pass to it references to functions that we
want to use for optimizing (local_optimization) and for creating a start configuration (create_start_particle)."""

guided_MC_search = GOS.GuidedSearch(local_optimization, create_start_particle)

"""We then have it fit the topological descriptors to the training set we just created"""
symbols = list(stoichiometry.keys()) #['Pt', 'Au']
guided_MC_search.fit_energy_expression(training_set, symbols)

"""We start the optimization by calling the run_multiple simulations function. In this case we want to do two
runs (n_sim_runs = 2). We can pass parameters to both functions (local optmization & create start particle) that will
be used when the optimization is actually started. In this case we make sure, that the start particle has the same
shape as the particles in the training set."""

n_sim_runs = 2
args_start = [5, 1, stoichiometry] # height, trunc, composition -> parameters of the create_start_particle function
results, run_times = guided_MC_search.run_multiple_simulations(n_sim_runs, args_start=args_start)

[-3.11475088e-01 -2.30459684e-02  1.09447395e+00  4.28830563e-01
 -1.90993430e-10  4.09461423e-14  2.95085946e-17 -7.65530154e-17
 -2.15645638e-16 -1.38586169e-29  9.86048765e-01  1.00990355e+00
 -1.39472284e-32  1.08115955e+00  0.00000000e+00  0.00000000e+00
  1.21119376e+00]
Coef symbol_a: Au
Run: 0
Runtime: 0.022996674990281463
Run: 1
Runtime: 0.021375728014390916


In [27]:
"""The results object will be a list of optimization runs. Each optimization run holds the final structure as well
as the energies and the step number of the considered configurations. Run times for every run will be returned as a
list"""
print('Structure : {}'.format(results[0][0]))
print(' ')
print('(Energy, step) pairs:')
print(results[0][1])
print(' ')
print('Runtime in s: {}'.format(run_times[0]))

Structure : <npl.core.nanoparticle.Nanoparticle object at 0x737a38eb0950>
 
(Energy, step) pairs:
[(28.638279344626003, 0), (28.30237473364122, 1), (27.992354807396445, 2), (27.688392841531304, 3), (27.416373520785804, 4), (27.10635359454103, 5), (26.8282763134159, 6), (26.664843379212904, 7), (26.492371702431118, 8), (26.358543161572907, 9), (26.294657871333623, 10), (26.189052266990284, 11), (26.15338991326587, 12), (26.073668993662544, 32), (26.019561372407658, 53), (26.019561372407658, 397)]
 
Runtime in s: 0.42455478198826313


In [28]:
"""Display the final structure of the first run"""
p = results[0][0]
view(p.get_ase_atoms(), viewer='x3d')

In [25]:
"""This is not the optimal solution. We can use the Basin Hopping instead of the local optimization to find it.
For this we have to import the run_basin_hopping function and pass some additional start parameters to the search
object"""

from BH.BasinHopping import run_basin_hopping

guided_MC_search = GOS.GuidedSearch(run_basin_hopping, create_start_particle)

"""Fit energy expression"""
symbols = list(stoichiometry.keys()) #['Pt', 'Au']
guided_MC_search.fit_energy_expression(training_set, symbols)

"""For the Basin Hopping we also have to pass the number of (not necessarily distinct) basins we want to expore 
(n_hopping_attempts) and how strong we want to perturbate the locally optimal solutions (n_exchanges). We can
pass them the same way as for the start particle. run_basin_hopping and local optimization also take different
parameters. Fortunately their function signatures are the same for the first three parameters, so we can reuse
the GuidedSearch class for both optmization functions. The remaining parameters can be passed simply as list, 
which should prove useful if the function signature changes at some point or one ones to implement a different
optimization procedure."""
n_hopping_attempts = 20
n_exchanges = 10
args_bh = [n_hopping_attempts, n_exchanges]

n_sim_runs = 2
args_start = [5, 1, stoichiometry] # height, trunc, composition -> parameters of the create_start_particle function

results, run_times = guided_MC_search.run_multiple_simulations(n_sim_runs, args_start=args_start, args_gm = args_bh)

[-3.11475088e-01 -2.30459684e-02  1.09447395e+00  4.28830563e-01
 -1.90993430e-10  4.09461423e-14  2.95085946e-17 -7.65530154e-17
 -2.15645638e-16 -1.38586169e-29  9.86048765e-01  1.00990355e+00
 -1.39472284e-32  1.08115955e+00  0.00000000e+00  0.00000000e+00
  1.21119376e+00]
Coef symbol_a: Au
Run: 0
Runtime: 0.42455478198826313
Run: 1
Runtime: 0.4381905280170031


In [26]:
"""Now we should have the best solution for this system, both runs finished with the same energy"""
p = results[0][0]
view(p.get_ase_atoms(), viewer='x3d')